# Predicting Heart Disease
#### Panoramica
- Obiettivo: Predict the likelihood of heart disease.
- Tipo di Competizione: Tabulare
- Metrica di Valutazione: Roc Auc Score
#### 
| Caratteristica | Descrizione |
| :--- | :---: |
| id | Identificativo unico per ogni paziente nel dataset. |
| Age | Età del paziente espressa in anni. |
| Sex | Genere del paziente (generalmente 1 = Maschio, 0 = Femmina). |
| Chest pain type | Tipologia di dolore toracico (da 1 a 4: anginoso tipico, atipico, non anginoso, asintomatico). |
| BP | Pressione sanguigna a riposo (espressa in mm Hg al momento dell'ammissione). |
| Cholesterol | Livello di colesterolo sierico espresso in mg/dl. |
| FBS over 120 | Zucchero nel sangue a digiuno > 120 mg/dl (1 = Vero; 0 = Falso). |
| EKG results | Risultati dell'elettrocardiogramma a riposo (valori 0, 1, 2 basati su anomalie). |
| Max HR | Frequenza cardiaca massima raggiunta durante il test da sforzo. |
| Exercise angina | Angina indotta dall'esercizio fisico (1 = Sì; 0 = No). |
| ST depression | Depressione del segmento ST indotta dall'esercizio rispetto al riposo. |
| Slope of ST | Pendenza del segmento ST nel picco dell'esercizio (1: crescente, 2: piatto, 3: decrescente). |
| Number of vessels fluro | Numero di vasi principali (0-3) colorati mediante fluoroscopia. |
| Thallium | Risultato del test al tallio (3 = normale; 6 = difetto fisso; 7 = difetto reversibile). |
| Heart Disease | Target: Presenza (1 o >0) o assenza (0) di patologia cardiaca. |


## 0. Environment & Data Fetching
(Configurazione, rilevamento ambiente, caricamento dataset)

In [ ]:
## 0. Environment & Data Fetching (Universal Setup)
import os
import pandas as pd
import warnings

COMPETITION_NAME = 'playground-series-s6e2' 
warnings.filterwarnings('ignore')

if os.getenv('KAGGLE_KERNEL_RUN_TYPE'):
    print(f"☁️  Rilevato ambiente Kaggle.")
    
    PATH = f'/kaggle/input/{COMPETITION_NAME}'
    
else:
    print("💻 Rilevato ambiente Locale.")
    PATH = '.' 
    
    import zipfile
    from kaggle.api.kaggle_api_extended import KaggleApi
    
    if not os.path.exists(f'{PATH}/train.csv'):
        print(f"⬇️  File non trovati. Avvio download per: {COMPETITION_NAME}...")
        
        try:
            api = KaggleApi()
            api.authenticate() 
            api.competition_download_files(COMPETITION_NAME, path=PATH)
            
            print("📦 Estrazione in corso...")
            with zipfile.ZipFile(f'{PATH}/{COMPETITION_NAME}.zip', 'r') as zip_ref:
                zip_ref.extractall(PATH)
            print("✅ Download ed estrazione completati!")
            
        except Exception as e:
            print(f"❌ Errore nel download (Verifica il file kaggle.json): {e}")
    else:
        print("✅ Dati già presenti in locale.")

try:
    train = pd.read_csv(f'{PATH}/train.csv')
    test = pd.read_csv(f'{PATH}/test.csv')
    sample_sub = pd.read_csv(f'{PATH}/sample_submission.csv')
    
    print(f"\n--- Data Loaded Successfully ---")
    print(f"Train shape: {train.shape}")
    print(f"Test shape:  {test.shape}")
    print(f"Sub shape:   {sample_sub.shape}")

    display(train.head(3)) 
    
except FileNotFoundError:
    print("\n❌ ERRORE CRITICO: Dataset non trovato!")
    print(f"Percorso cercato: {PATH}")

## 1. Imports & Global Configuration
(Importazione librerie, configurazione Plotly e Seed per la riproducibilità)

In [ ]:
import numpy as np

# Visualization 
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt 
import seaborn as sns

# Scikit-Learn Preprocessing & Validation
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, ClassifierMixin

# Scikit-Learn Metrics
from sklearn.metrics import roc_auc_score, auc, accuracy_score, confusion_matrix, classification_report
import plotly.figure_factory as ff

# Models
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Configuration
warnings.filterwarnings('ignore') 
pio.templates.default = "plotly_white" 

# Global Constants
RANDOM_STATE = 42 
N_FOLDS = 10 

print("✅ Librerie importate.")

## 2. Exploratory Data Analysis (EDA)
(Analisi distribuzioni, correlazioni, outlier, visualizzazione target)

In [ ]:
## 2.X Data Cleaning & Target Fix

print("\n--- DIAGNOSTICA PRE-FIX ---")
print(f"Valori unici in 'Heart Disease': {train['Heart Disease'].unique()}")
print(f"Tipo di dato attuale: {train['Heart Disease'].dtype}")

# FIX: Se il tipo è object (stringa) o contiene valori testuali, li riconvertiamo
if train['Heart Disease'].dtype == 'object' or 'Absence' in train['Heart Disease'].values:
    print("⚠️ Rilevate stringhe nel Target. Conversione in numeri (0/1)...")
    
    mapping_fix = {
        'Absence': 0, 
        'Presence': 1
    }
    
    train['Heart Disease'] = train['Heart Disease'].map(mapping_fix)

train['Heart Disease'] = pd.to_numeric(train['Heart Disease'], errors='coerce').fillna(0).astype(int)

print("\n--- DIAGNOSTICA POST-FIX ---")
print(f"✅ Valori unici finali: {train['Heart Disease'].unique()}")
print(f"✅ Tipo di dato finale: {train['Heart Disease'].dtype}")

In [ ]:
# --- 2.1 Data Health Check ---

print(f"Valori mancanti nel Train: {train.isnull().sum().sum()}")
print(f"Valori mancanti nel Test:  {test.isnull().sum().sum()}")

duplicates = train.duplicated().sum()
if duplicates > 0:
    print(f"⚠️ Trovati {duplicates} duplicati nel training set. Considera di rimuoverli.")
else:
    print("✅ Nessun duplicato trovato.")

# --- 2.2 Target Distribution ---
target_counts = train['Heart Disease'].value_counts().reset_index()
target_counts.columns = ['Heart Disease', 'Count']
target_counts['Label'] = target_counts['Heart Disease'].astype(int).map({0: 'No Disease', 1: 'Disease'})

fig = px.pie(target_counts, 
             values='Count', 
             names='Label', 
             title='Distribuzione Target: Presenza vs Assenza Malattia Cardiaca',
             color='Label',
             color_discrete_map={'No Disease':'#66b3ff', 'Disease':'#ff9999'},
             hole=0.4) 

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
# --- 2.3 Numerical Features Distribution ---

numerical_cols = ['Age', 'BP', 'Cholesterol', 'Max HR', 'ST depression', 'Number of vessels fluro']

for col in numerical_cols:
    fig = px.histogram(train, 
                       x=col, 
                       color='Heart Disease',
                       marginal="box", 
                       title=f'Distribuzione di {col} rispetto al Target',
                       color_discrete_map={0:'#66b3ff', 1:'#ff9999'},
                       opacity=0.7,
                       barmode='overlay') 
    
    fig.update_layout(bargap=0.1)
    fig.show()

In [ ]:
# --- 2.4 Categorical Features Analysis ---

cat_cols = ['Sex', 'Chest pain type', 'FBS over 120', 'EKG results', 
            'Exercise angina', 'Slope of ST', 'Thallium']

for col in cat_cols:
    df_grouped = train.groupby(col)['Heart Disease'].value_counts(normalize=True).rename('percentage').reset_index()
    df_grouped['percentage'] = df_grouped['percentage'] * 100
    df_grouped['Heart Disease'] = df_grouped['Heart Disease'].astype(int).map({0: 'Absence', 1: 'Presence'})
    
    df_grouped = df_grouped.sort_values(by=[col, 'Heart Disease'])

    fig = px.bar(df_grouped, 
                 x=col, 
                 y='percentage', 
                 color='Heart Disease',
                 barmode='group',
                 title=f'Incidenza Malattia per: {col}',
                 color_discrete_map={'Absence':'#66b3ff', 'Presence':'#ff9999'},
                 text_auto='.1f') 
    
    fig.update_layout(yaxis_title="Percentuale (%)")
    fig.show()

In [ ]:
# --- 2.5 Correlation Matrix ---

corr_matrix = train.corr(numeric_only=True)

fig = px.imshow(corr_matrix, 
                text_auto='.2f', 
                aspect="auto",
                color_continuous_scale='RdBu_r', 
                title='Matrice di Correlazione (Heatmap)')

fig.show()

Sex (Bar Chart)
Il "Gender Gap": C'è una differenza enorme.
- Donne (0): Solo il 17.9% ha la malattia. Essere donna è un forte fattore protettivo in questo dataset.
- Uomini (1): Il rischio sale al 55.6%. Più della metà degli uomini nel dataset presenta la patologia.

Chest Pain Type (Bar Chart)
Il Paradosso dell'Asintomatico:
- I tipi 1, 2 e 3 hanno prevalenza di "Sani" (Barre blu alte).
- Il Tipo 4 (Asintomatico) è critico: quasi il 70% (69.7%) dei pazienti in questa categoria è malato. Se il modello vede "Type 4", alzerà drasticamente la probabilità di rischio.

- Exercise Angina: Questa è una Red Flag. Se c'è angina durante lo sforzo (1), la probabilità di malattia schizza verso l'alto (barra turchese molto ridotta rispetto alla controparte).

ST Depression (Box Plot)
Il marcatore dell'ischemia:
- I sani sono concentrati sullo 0.
- Appena il valore sale sopra lo 0.5 o 1.0, la probabilità di malattia domina. La "coda" rossa verso destra è un segnale inequivocabile.

- Thallium: Il valore 3 sembra "sicuro" (Normal), mentre 7 (Reversibile) e 6 (Fisso) portano con sé un alto tasso di positività.

Max HR (Box Plot + Histogram)
La prova da sforzo:
- Si nota una separazione netta. I pazienti sani (Blu) riescono a spingere il cuore a frequenze molto più alte (mediana intorno a 160 bpm).
- I pazienti malati (Rosso) si fermano prima (mediana intorno a 130-140 bpm).

Number of Vessels Fluro (Bar Chart)
Gradino di rischio:
- 0 Vasi: La stragrande maggioranza è sana.
- 1, 2, 3 Vasi: La situazione si ribalta completamente. Avere anche solo un vaso colorato dalla fluoroscopia indica una probabilità altissima di malattia.

- Correlazioni: La Heatmap conferma che ST depression, Exercise angina e Number of vessels sono i predittori positivi più forti. Max HR è il più forte predittore negativo (più alto è il battito massimo, più sano è il cuore).

## 3. Data Preprocessing & Feature Engineering
(Gestione valori mancanti, encoding categoriche, scaling, creazione nuove feature)

In [ ]:
## 3. Data Preprocessing & Feature Engineering

# --- 3.1 Feature Creation Function ---
def create_features(df):
    df = df.copy()
    
    df['Cholesterol_Age_Ratio'] = df['Cholesterol'] / df['Age']
    
    df['Age_MaxHR_Interaction'] = df['Age'] * df['Max HR']

    df['Hemodynamic_Risk'] = df['Age'] * df['BP']
    
    df['Severe_Angina'] = ((df['Chest pain type'] == 4) & (df['Exercise angina'] == 1)).astype(int)

    df['Angina_Thallium_Combo'] = df['Exercise angina'] * df['Thallium']
    
    df['ST_Heart_Stress'] = df['ST depression'] * df['Slope of ST']
    
    df['Age_Group'] = pd.cut(df['Age'], bins=[0, 45, 60, 100], labels=[0, 1, 2]).astype(int)
    
    return df

train_eng = create_features(train)
test_eng = create_features(test)

print("✅ Feature Engineering completato. Nuove colonne create.")
display(train_eng[['Cholesterol_Age_Ratio', 'Age_MaxHR_Interaction', 'Hemodynamic_Risk', 
                   'Severe_Angina', 'Angina_Thallium_Combo', 'ST_Heart_Stress', 'Age_Group']].head(4))

# --- 3.2 Preprocessing Pipeline Setup ---

target_col = 'Heart Disease'

categorical_features = [
    'Sex', 'Chest pain type', 'FBS over 120', 'EKG results', 
    'Exercise angina', 'Slope of ST', 'Thallium', 
    'Age_Group', 'Severe_Angina' 
]

numeric_features = [
    col for col in train_eng.columns 
    if col not in categorical_features + ['id', target_col]
]

print(f"\n🔢 Feature Numeriche ({len(numeric_features)}): {numeric_features}")
print(f"🔠 Feature Categoriche ({len(categorical_features)}): {categorical_features}")

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), 
    ('scaler', StandardScaler()) 
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)) 
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    verbose_feature_names_out=False 
)

preprocessor.set_output(transform='pandas')

X = train_eng.drop(columns=['id', target_col])
y = train_eng[target_col]

X_test = test_eng.drop(columns=['id']) 
test_ids = test_eng['id']

print("\n⚙️ Preprocessor configurato.")
print(f"✅ Dati pronti: X shape: {X.shape}, y shape: {y.shape}")

new_feats = ['Cholesterol_Age_Ratio', 'Age_MaxHR_Interaction', 'Hemodynamic_Risk', 
             'Severe_Angina', 'Angina_Thallium_Combo', 'ST_Heart_Stress']
print("\n📊 Correlazione Nuove Feature con il Target:")
print(train_eng[new_feats + [target_col]].corr()[target_col].sort_values(ascending=False))

## 4. Hardware-Aware Model Definition & Selection
(Definizione modelli adattiva: GPU per Kaggle, CPU Ottimizzata per Mac/PC, pipeline e strategia di validazione)

In [ ]:
## 4. Model Selection & Evaluation Strategy

# --- 4.1 Model Definitions ---

import shutil

HAS_NVIDIA_GPU = shutil.which('nvidia-smi') is not None
IS_KAGGLE = os.getenv('KAGGLE_KERNEL_RUN_TYPE') is not None

print(f"⚙️ Hardware Detection: {'✅ GPU NVIDIA Trovata (Mode: FAST)' if HAS_NVIDIA_GPU else '💻 GPU NVIDIA non trovata (Mode: CPU Compatibility)'}")

if HAS_NVIDIA_GPU:
    print("🚀 Configurazione GPU Attiva (CUDA)")
    xgb_params = {
        'n_estimators': 1000, 'learning_rate': 0.05, 'max_depth': 6,
        'subsample': 0.8, 'colsample_bytree': 0.8, 'random_state': RANDOM_STATE,
        'eval_metric': 'auc', 'device': 'cuda', 'tree_method': 'hist'
    }
    lgbm_params = {
        'n_estimators': 1000, 'learning_rate': 0.05, 'num_leaves': 31,
        'random_state': RANDOM_STATE, 'verbose': -1, 'device': 'gpu'
    }
    cat_params = {
        'iterations': 1000, 'learning_rate': 0.05, 'depth': 6,
        'random_seed': RANDOM_STATE, 'verbose': 0, 'allow_writing_files': False,
        'task_type': 'GPU', 'devices': '0'
    }
    CV_N_JOBS = 1

else:
    print("⚠️ Configurazione CPU (Compatibilità Mac/PC)")
    
    xgb_params = {
        'n_estimators': 1000, 'learning_rate': 0.05, 'max_depth': 6,
        'subsample': 0.8, 'colsample_bytree': 0.8, 'random_state': RANDOM_STATE,
        'eval_metric': 'auc',
        'n_jobs': 4 
    }
    lgbm_params = {
        'n_estimators': 1000, 'learning_rate': 0.05, 'num_leaves': 31,
        'random_state': RANDOM_STATE, 'verbose': -1,
        'n_jobs': 4
    }
    cat_params = {
        'iterations': 1000, 'learning_rate': 0.05, 'depth': 6,
        'random_seed': RANDOM_STATE, 'verbose': 0,
        'allow_writing_files': False, 
        'loss_function': 'Logloss',  
        'eval_metric': 'AUC' 
    }
    CV_N_JOBS = 1

models = {
    'XGBoost': XGBClassifier(**xgb_params),
    'LightGBM': LGBMClassifier(**lgbm_params),
    'CatBoost': CatBoostClassifier(**cat_params) 
}

# --- 4.2 Cross-Validation Function ---

def evaluate_models(models, X, y, preprocessor):
    results = {}
    print(f"\n🚀 Inizio Training su {len(X)} righe con {N_FOLDS}-Fold CV...")
    
    for name, model in models.items():
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', model)
        ])
        
        cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
        
        try:
            scores = cross_val_score(pipeline, X, y, cv=cv, scoring='roc_auc', n_jobs=CV_N_JOBS, error_score='raise')
            mean_score = np.mean(scores)
            print(f"✅ {name:10} | ROC-AUC: {mean_score:.5f} (+/- {np.std(scores):.5f})")
            results[name] = mean_score
        except Exception as e:
            print(f"❌ {name:10} | ERRORE: {e}")
            results[name] = 0 
        
    return results

# --- 4.3 Execute Evaluation ---
model_scores = evaluate_models(models, X, y, preprocessor)
best_model_name = max(model_scores, key=model_scores.get)
print(f"\n🏆 Best Model: {best_model_name} con AUC: {model_scores[best_model_name]:.5f}")

## 5. Ensemble Construction & Training
(Creazione del Voting Classifier e configurazione dell'addestramento)

In [ ]:
## 5. Ensemble & Final Training

print("🤝 Costruzione dell'Ensemble (XGBoost + LightGBM + CatBoost)...")

successful_models = [name for name, score in model_scores.items() if score > 0.5]

if not successful_models:
    raise ValueError("❌ Nessun modello valido trovato! Controlla gli errori sopra.")

estimators_list = []
print(f"Modelli inclusi nell'Ensemble: {successful_models}")

if 'XGBoost' in successful_models:
    estimators_list.append(('xgb', Pipeline(steps=[('preprocessor', preprocessor), ('model', XGBClassifier(**xgb_params))])))
if 'LightGBM' in successful_models:
    estimators_list.append(('lgbm', Pipeline(steps=[('preprocessor', preprocessor), ('model', LGBMClassifier(**lgbm_params))])))
if 'CatBoost' in successful_models:
    estimators_list.append(('cat', Pipeline(steps=[('preprocessor', preprocessor), ('model', CatBoostClassifier(**cat_params))])))

ensemble_model = VotingClassifier(
    estimators=estimators_list,
    voting='soft',
    n_jobs=1 
)

print(f"🚀 Validazione Ensemble ({len(estimators_list)} modelli)...")
ensemble_scores = cross_val_score(ensemble_model, X, y, cv=10, scoring='roc_auc', n_jobs=CV_N_JOBS)

mean_ens = np.mean(ensemble_scores)
std_ens = np.std(ensemble_scores)

print(f"🏆 Ensemble ROC-AUC: {mean_ens:.5f} (+/- {std_ens:.5f})")

print("\n💪 Addestramento finale sul 100% dei dati di Train...")
ensemble_model.fit(X, y)
print("✅ Training completato.")

## 6. Model Diagnostics & Visual Evaluation
(Analisi approfondita: Curve ROC, Matrice di Confusione e Report di Classificazione)

In [ ]:
## 6. Model Diagnostics & Visual Evaluation
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
import plotly.figure_factory as ff

print("🔍 Esecuzione Diagnostica (questo potrebbe richiedere un attimo)...")

y_train_pred_proba = cross_val_predict(ensemble_model, X, y, cv=10, method='predict_proba', n_jobs=4)[:, 1]

fpr, tpr, thresholds = roc_curve(y, y_train_pred_proba)
roc_auc = auc(fpr, tpr)

# --- A. ROC Curve (Plotly) ---
fig_roc = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC = {roc_auc:.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=600
)
fig_roc.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)
fig_roc.update_yaxes(scaleanchor="x", scaleratio=1)
fig_roc.update_xaxes(constrain='domain')
fig_roc.show()

# --- B. Confusion Matrix ---
threshold = 0.5
y_train_pred_class = (y_train_pred_proba > threshold).astype(int)

cm = confusion_matrix(y, y_train_pred_class)

z = cm
x = ['Predicted Healthy', 'Predicted Disease']
y_labels = ['Actual Healthy', 'Actual Disease']

# Standard sklearn:
# [[TN, FP],
#  [FN, TP]]

fig_cm = ff.create_annotated_heatmap(
    z, x=x, y=y_labels, colorscale='Blues', showscale=True
)
fig_cm.update_layout(title_text='Confusion Matrix (Threshold = 0.5)', width=600, height=500)
fig_cm.show()

# --- C. Classification Report ---
print("\n📝 Report di Classificazione Dettagliato:")
print(classification_report(y, y_train_pred_class, target_names=['Healthy', 'Disease']))

## 7. Final Prediction & Submission
(Inferenza sul Test Set, check della distribuzione e creazione CSV)

In [ ]:
## 7. Final Prediction & Submission

# --- 7.1 Prediction on Test Set ---
print("🔮 Generazione predizioni sul Test Set...")

y_pred_probs = ensemble_model.predict_proba(X_test)[:, 1]

# --- 7.2 Submission DataFrame Creation ---
submission = pd.DataFrame({
    'id': test_ids,
    'Heart Disease': y_pred_probs
})

print("\n--- Anteprima Submission ---")
display(submission.head())

fig = px.histogram(submission, x='Heart Disease', nbins=50, title='Distribuzione Probabilità Predette (Test Set)')
fig.show()

# --- 7.3 Save to CSV ---
file_name = 'submission.csv'
submission.to_csv(file_name, index=False)

print(f"✅ File salvato: {file_name}")
print(f"Dimensione file: {submission.shape}")

from IPython.display import FileLink
FileLink(file_name)